In [1]:
import pypokedex
import requests
import json
import pandas as pd
import re
from sqlalchemy import create_engine, text
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Float 
from sqlalchemy.orm import Session

In [2]:
def get_id(url):
    return re.search('([0-9]+)/$', url).group(1)

In [3]:
# URL for GET requests to retrieve generation (games) data
url = "https://pokeapi.co/api/v2/generation/"

In [4]:
# Print the response object to the console
generations = requests.get(url).json()

In [5]:
# Retrieving data and converting it into JSON
generations = generations['results']
generations

[{'name': 'generation-i', 'url': 'https://pokeapi.co/api/v2/generation/1/'},
 {'name': 'generation-ii', 'url': 'https://pokeapi.co/api/v2/generation/2/'},
 {'name': 'generation-iii', 'url': 'https://pokeapi.co/api/v2/generation/3/'},
 {'name': 'generation-iv', 'url': 'https://pokeapi.co/api/v2/generation/4/'},
 {'name': 'generation-v', 'url': 'https://pokeapi.co/api/v2/generation/5/'},
 {'name': 'generation-vi', 'url': 'https://pokeapi.co/api/v2/generation/6/'},
 {'name': 'generation-vii', 'url': 'https://pokeapi.co/api/v2/generation/7/'},
 {'name': 'generation-viii', 'url': 'https://pokeapi.co/api/v2/generation/8/'},
 {'name': 'generation-ix', 'url': 'https://pokeapi.co/api/v2/generation/9/'}]

In [6]:
gen_url = generations[0]['url'].strip()


In [7]:
pokemon_species = requests.get(gen_url).json()['pokemon_species']
pokemans = []
for pokemon in pokemon_species:
    pokemans.append(get_id(pokemon['url']))

print(len(pokemans))

151


In [8]:
# Fill out the Games table by finding all Pokemon IDs
game_id = []
game_name = []
# loop through each generation
for generation in generations:
    gen_url = generation['url']
    gen_json = requests.get(gen_url).json()
    # to track which games a generation can be in, need to drill down into version group
    for version_group in gen_json['version_groups']:
        grp_url = version_group['url']
        grp_json = requests.get(grp_url).json()
        # keep track of each individual game
        for game in grp_json['versions']:
            game_id.append(get_id(game['url']))
            game_name.append(game['name'])

# create DataFrame to store pokemon games
game_df = pd.DataFrame({
    'game_id':game_id,
    'game_name':game_name
})

game_df['game_id'] = game_df['game_id'].astype(int)


In [9]:
game_df

,game_id,game_name
0,1,red
1,2,blue
2,3,yellow
3,4,gold
4,5,silver
5,6,crystal
6,7,ruby
7,8,sapphire
8,9,emerald
9,10,firered


In [10]:
# Fill out the pokeGame table by finding all Pokemon IDs
poke_id = []
game_id = []
# loop through each generation
for generation in generations:
    gen_url = generation['url']
    gen_json = requests.get(gen_url).json()
    available_in = []
    # to track which games a generation can be in, need to drill down into version group
    for version_group in gen_json['version_groups']:
        grp_url = version_group['url']
        grp_json = requests.get(grp_url).json()
        # keep track of each individual game
        for game in grp_json['versions']:
            available_in.append(get_id(game['url']))

    # for each pokemon in the pokemon_species list
    for pokemon in gen_json['pokemon_species']:
        id = get_id(pokemon['url'])
        # add an entry for each game that the pokemon could be in
        for game in available_in:
            poke_id.append(id)
            game_id.append(game)

# create DataFrame to store pokemon games
pokegame_df = pd.DataFrame({
    'poke_id':poke_id,
    'game_id':game_id
})

pokegame_df[['game_id', 'poke_id']] = pokegame_df[['game_id', 'poke_id']].astype(int)

In [11]:
pokegame_df

,poke_id,game_id
0,1,1
1,1,2
2,1,3
3,4,1
4,4,2
...,...,...
4820,1018,43
4821,1019,40
4822,1019,41
4823,1019,42


In [12]:
engine = create_engine(f'sqlite:///Pokemon.sqlite')

game_df.to_sql('game', engine)
pokegame_df.to_sql('pokeGame', engine)

4825

In [13]:
engine.dispose()